In [171]:
import numpy as np
from scipy.optimize import root
from scipy.integrate import odeint
from scipy.linalg import solve_continuous_are as scare
import gym, acme_gym

In [172]:
#These are the functions from Brooke's Inverted Pendulum lab
def linearized_init(M, m, l, q1, q2, q3, q4, r):
    '''
    Parameters:
    ----------
    M, m: floats
    masses of the rickshaw and the present
    l   : float
    length of the rod
    q1, q2, q3, q4, r : floats
    relative weights of the position and velocity of the rickshaw, 
    the angular displacement theta and the change in theta, and the control
    Return
    -------
    A : ndarray of shape (4,4)
    B : ndarray of shape (4,1)
    Q : ndarray of shape (4,4)
    R : ndarray of shape (1,1)
    '''
    g = 9.8
    A = np.zeros((4,4))
    A[0,1] = 1
    A[1,2] = m*g/M
    A[2,3] = 1
    A[3,2] = g/(M*l)*(M+m)
    B = np.zeros((4,1))
    B[1] = 1/M
    B[3] = 1/(M*l)
    Q = np.diag([q1, q2, q3, q4])
    R = np.array([[r]])
    return A, B, Q, R

def find_P(A, B, Q, R):
    '''
    Parameters:
    ----------
    A, Q    : ndarrays of shape (4,4)
    B       : ndarray of shape (4,1)
    R       : ndarray of shape (1,1)
    Returns
    -------
    P       : the matrix solution of the Riccati equation
    '''
    def fun(P):
        P = P.reshape((4,4))
        root = P@A+A.T@P+Q-1/R[0]*(P@B@B.T@P)
        return root.reshape(16)
    P0 = np.ones(16)
    P = root(fun,P0).x.reshape((4,4))
    return P
def rickshaw(tv, X0, A, B, Q, R, P):
    '''
    Parameters:
    ----------
    tv  : ndarray of time values, with shape (n+1,)
    X0  : Initial conditions on state variables
    A, Q: ndarrays of shape (4,4)
    B   : ndarray of shape (4,1)
    R   : ndarray of shape (1,1)
    P   : ndarray of shape (4,4)
    Returns
    -------
    Z : ndarray of shape (n+1,4), the state vector at each time
    U : ndarray of shape (n+1,), the control values
    '''
    
    func = lambda z,t: (A - 1/R[0]*B@B.T@P)@z.T
    Z = odeint(func, X0, tv)
    BP = B.T@P
    U = -1/R[0]*(BP)@Z.T
    return Z,U  

In [173]:
env.close()#always close previous environment before opening new one
#This is the new function that will take in information for inverted 
#pendulum problem, and will find optimal solution and display on
#environment given
def stabilize(M, m, l, q1, q2, q3, q4, r, X0, tf, step):
    A, B, Q, R = linearized_init(M, m, l, q1, q2, q3, q4, r)
    P = scare(A, B, Q, R)
    tv = np.linspace(0,tf,step)
    Z, U = rickshaw(tv,X0,A, B, Q, R, P)
    #Z is state vector
    #U is control values
    return Z,U
step = 300
env = gym.make("CartPoleContinuous-v0")
obs = env.reset()
x0 = obs
q1, q2, q3, q4 = 1., 1., 5., 2
env.render()
tf = .02*step

r = 10. # Weight on the control, how do we know what it should be?
M, m, l = 1, .1, 1
Z, U = stabilize(M, m, l, q1, q2, q3, q4, r, x0, tf, step)
U = U.T

for i in range(step):
    #get a new obs every interation... are we supposed to include that in new calcs?
    obs, reward, state, info = env.step(np.array(U[0]))
    env.render()
                                        
    # Feedback
    x0 = np.array([obs[0],obs[1],-obs[2],-obs[3]])
    Z, U = stabilize(M, m, l, q1, q2, q3, q4, r, x0, tf, step)
    U = U.T

In [16]:
env.close()

In [174]:
def easy_run(x0, q1, q2, q3, q4, r, env, step = 200):
    states = []
    tf = .02*step
    M, m, l = 1, .1, 1
    for i in range(step):
        Z, U = stabilize(M, m, l, q1, q2, q3, q4, r, x0, tf, step)
        U = U.T
        #get a new obs every interation... are we supposed to include that in new calcs?
        obs, reward, state, info = env.step(np.array(U[0]))
        states.append(state)
        # Feedback
        x0 = np.array([obs[0],obs[1],-obs[2],-obs[3]])
        if np.sum(np.array(states[-5:]).astype(int)) > 4:
            break
    return len(states)

In [ ]:
env1 = gym.make("CartPoleContinuous-v0")
env2 = gym.make("CartPoleContinuous-v0")
env3 = gym.make("CartPoleContinuous-v0")
env4 = gym.make("CartPoleContinuous-v0")
env5 = gym.make("CartPoleContinuous-v0")
diff_envs = [env1, env2, env3, env4, env5]
best_iters = np.inf
best_vals = None
for q1 in np.linspace(0,5,11):
    for q2 in np.linspace(.5,5,10):
        for q3 in np.linspace(.5,5,10):
            for q4 in np.linspace(.5,5,10):
                for r in np.linspace(.5,5,10):
                    sum_iters = 0
                    for env in diff_envs:
                        sum_iters += easy_run(env.reset(), q1, q2, q3, q4, r, env)
                    if sum_iters < best_iters:
                        best_iters = sum_iters
                        best_vals = [q1, q2, q3, q4, r]

In [ ]:
print(best_iters,'\n',best_vals)